In [ ]:
!git clone https://github.com/neonbjb/tortoise-tts.git
%cd tortoise-tts
!pip3 install -r requirements.txt
!python3 setup.py install


Cloning into 'tortoise-tts'...
remote: Enumerating objects: 1498, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 1498 (delta 0), reused 1 (delta 0), pack-reused 1494
Receiving objects: 100% (1498/1498), 53.58 MiB | 8.46 MiB/s, done.
Resolving deltas: 100% (595/595), done.
/content/tortoise-tts
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 77.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 102.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 KB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 84.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 98.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [ ]:
# Imports used through the rest of the notebook.
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F

import IPython

from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices

# This will download all the models used by Tortoise from the HF hub.
tts = TextToSpeech()

Downloading:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/159 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/181 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Done.


Done.


Done.


Done.


In [44]:
# This is the text that will be spoken.
text = """This section helps you categorize your problem in order to realize the role of AI/ML in finding a solution to your business problem. 
Which bucket does your problem fall under? 
A First Principles problem can be realized when an existing approach to address an issue can be found.  In short, there exists a known methodology to solve the problem. Methodical disassembling of a computer to repair its defective parts may be considered as a first principles problem. 
"""#The emotional, strategic, legal, and ethical bucket of problems are typically found in your business. Picking an ideal applicant for employment, addressing a customer churn problem, avoiding discrimination and bias, conducting business within your personal & professional morals and values some business scenarios that fall under this bucket. """
# A logical or a rational business problem is that which you know has reason but not know of an approach towards its solution.  
# Ensure to categorize your business problem into one of these buckets, divide the problem further, if necessary. This helps to optimize your AI/ML approach to its best potential.  
# If your organization is in the beginning phases of adapting AI, we recommend for you to pick problems that fall under the third bucket.
# Why? """
# For solving the first bucket, AI/ML mostly aid in minimizing the number of choices or parameters required by a human expert to execute the known optimal approach. The second bucket requires multiple human experts and AI/ML systems to find an optimal solution. However, AI/ML systems are just aids for the experts.
# The third bucket is where an AI/ML system is your front-end problem-solving contributor. Predicting a stock price can be a good example for this bucket. Picking third-bucket problems will thus help you immediately realize the value of AI/ML systems.
# If there is insufficient data for you to incorporate the systems, focus on collecting the data.  
# For any third-bucket problem, it is efficient to quickly incorporate AI/ML systems for problem-solving."""

# Pick a "preset mode" to determine quality. Options: {"ultra_fast", "fast" (default), "standard", "high_quality"}. See docs in api.py
preset = "standard"

%ls tortoise/voices

IPython.display.Audio('tortoise/voices/tom/1.wav')


In [16]:
import wave, os, glob

In [24]:
zero=[]
path1='/content/tortoise-tts/tortoise/voices'
for filename in os.listdir(path):
  path2=path1+'/'+str(filename)
  for file in glob.glob(os.path.join(path2,"*.wav")):
    zero.append(file)


In [42]:
IPython.display.Audio(zero[5])

In [ ]:
for i in IPython.display.Audio('tortoise/voices/deniro/1.wav')

In [ ]:
# Pick one of the voices from the output above
voice = 'applejack'

# Load it and send it through Tortoise.
voice_samples, conditioning_latents = load_voice(voice)
gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents, 
                          preset=preset)
torchaudio.save('generated.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio('generated.wav')

In [ ]:
# Tortoise can also generate speech using a random voice. The voice changes each time you execute this!
# (Note: random voices can be prone to strange utterances)
gen = tts.tts_with_preset(text, voice_samples=None, conditioning_latents=None, preset=preset)
torchaudio.save('generated.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio('generated.wav')

In [ ]:
# You can also combine conditioning voices. Combining voices produces a new voice
# with traits from all the parents.
#
# Lets see what it would sound like if Picard and Kirk had a kid with a penchant for philosophy:
voice_samples, conditioning_latents = load_voices(['pat', 'william'])

gen = tts.tts_with_preset("They used to say that if man was meant to fly, he’d have wings. But he did fly. He discovered he had to.", 
                          voice_samples=None, conditioning_latents=None, preset=preset)
torchaudio.save('captain_kirkard.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio('captain_kirkard.wav')

In [ ]:
del tts  # Will break other cells, but necessary to conserve RAM if you want to run this cell.

# Tortoise comes with some scripts that does a lot of the lifting for you. For example,
# read.py will read a text file for you.
!python3 tortoise/read.py --voice=train_atkins --textfile=tortoise/data/riding_hood.txt --preset=ultra_fast --output_path=.

IPython.display.Audio('train_atkins/combined.wav')
# This will take awhile..